## Config

In [2]:
import matplotlib.pyplot as plt

In [3]:
%matplotlib inline

In [4]:
from numpy import outer, eye, ones, zeros, diag, log, sqrt, exp, pi
from numpy.linalg import inv, solve
from numpy.random import multivariate_normal as mvnormal, normal, gamma, beta, binomial
from scipy.special import gammaln
from scipy.stats import norm, multivariate_normal

from numpy import zeros
from numpy.random import randn

import numpy as np

import matplotlib.pyplot as plt
from numpy import arange, min, max, sqrt, mean, std
from scipy.spatial.distance import cosine

import copy

## EM-algorithm

In [5]:
# Aalto University, School of Science
# T-61.5140 Machine Learning: Advanced probabilistic Methods
# Author: antti.kangasraasio@aalto.fi, 2016

class EM_algo():
    """
        A superclass for different EM-fitted models.
    """

    def __init__(self, hyperparams, X=None, Y=None, ndata=0, pdata=0):
        """
            Initialize model based either on given data (X, Y) or
            on given data dimensionality (ndata, pdata).
        """
        if not X is None and not Y is None:
            self.X = X
            self.Y = Y
            self.ndata = len(self.X)
            self.pdata = len(self.X[0])
        if ndata and pdata:
            self.X = None
            self.Y = None
            self.ndata = ndata
            self.pdata = pdata
        self.h = hyperparams
        self.p = dict() # model parameters
        self.reset()
        if not X is None and not Y is None:
            self.current_logl, self.cll = self.logl()


    def reset(self):
        """
            Reset priors and draw parameter estimates from prior.
        """
        raise NotImplementedError("Subclass implements")


    def draw(self, item):
        """
            Draw a data sample from the current predictive distribution.
            Returns the drawn y and z-values.
        """
        raise NotImplementedError("Subclass implements")


    def logl(self):
        """
            Calculates the full log likelihood for this model.
            Returns the logl (and the values of each term for debugging purposes)
        """
        raise NotImplementedError("Subclass implements")


    def EM_iter(self):
        """
            Executes a single round of EM updates for this model.
        """
        raise NotImplementedError("Subclass implements")


    def EM_fit(self, alim=1e-10, maxit=1e4):
        """
            Calls the EM_iter repeatedly until the log likelihood
            of the model increases less than 'alim' in absolute
            value or after 'maxit' iterations have been done.

            Returns the number of EM-iterations, final log likelihood
            value and a string that explains the end condition.
        """
        logl, ll = self.logl()
        for i in range(int(maxit)):
            self.EM_iter()
            logl2, ll2 = self.logl()
            adiff = abs(logl2 - logl)
            if adiff < alim:
                return i+1, logl2, "alim"
            logl = logl2
        return maxit, logl2, "maxit"


    def assert_logl_increased(self, event):
        """
            Checks that the log likelihood increased since model
            initialization or the time this function was last called.
        """
        newlogl, ll = self.logl()
#         if self.current_logl - newlogl > 1e-3:
        if self.current_logl - newlogl > 0.1:
            self.debug_logl(self.cll, ll)
            raise ValueError("logl decreased after %s" % (event))
        self.current_logl, self.cll = newlogl, ll


    def get_p(self):
        """
            Returns a copy of the model parameters.
        """
        return copy.deepcopy(self.p)


    def set_p(self, p):
        """
            Sets the model parameters.
        """
        self.p = p.copy()


    def print_p(self):
        """
            Prints the model parameters, one at each line.
        """
        for k, v in self.p.items():
            print("%s = %s" % (k, v))


    def pretty_vector(self, x):
        """
            Returns a formatted version of a vector.
        """
        s = ["("]
        s.extend(["%.2f, " % (xi) for xi in x[:-1]])
        s.append("%.2f)" % (x[-1]))
        return "".join(s)


    def debug_logl(self, ll1, ll2):
        """
            Prints an analysis of the per-term change in
            log likelihood from ll1 to ll2.
        """
        print("Logl      before     after")
        for v1, v2, i in zip(ll1, ll2, range(len(ll1))):
            if v1 > v2:
                d = ">"
            elif v2 > v1:
                d = "<"
            else:
                d = "="
            print("Term %02d: %7.3f %s %7.3f" % (i, v1, d, v2))
        print("Total    %7.3f   %7.3f" % (sum(ll1), sum(ll2)))



## Linear model

In [16]:
# Aalto University, School of Science
# T-61.5140 Machine Learning: Advanced probabilistic Methods
# Author: antti.kangasraasio@aalto.fi, 2016

class EM_algo_LM(EM_algo):
    """
        A linear gaussian model.
    """

    def reset(self):
        """
            Reset priors and draw parameter estimates from prior.
        """
        # priors
        self.lbd_phi0       = self.h["lbd_phi0"]
        self.alpha_s20      = self.h["alpha_s20"]
        self.beta_s20       = self.h["beta_s20"]
        self.sigma_phi0     = eye(self.pdata) * self.h["lbd_phi0"]
        self.sigma_phi0_inv = eye(self.pdata) / self.h["lbd_phi0"]
        self.mu_phi0        = ones(self.pdata) * self.h["mu_phi0"]

        # initial parameter estimates drawn from prior
        self.p           = dict()
        self.p["sigma2"] = 1.0 / gamma(self.alpha_s20, 1.0 / self.beta_s20) # inverse gamma
        self.p["phi"]    = mvnormal(self.mu_phi0, self.p["sigma2"] * self.sigma_phi0)


    def draw(self, item):
        """
            Draw a data sample from the current predictive distribution.
            Returns the y-value (and a constant z-value for compatibility)
        """
        mean = float(item.dot(self.p["phi"]))
        std  = sqrt(self.p["sigma2"])
        return normal(mean, std), 1


    def logl(self):
        """
            Calculates the full log likelihood for this model.
            Returns the logl (and the values of each term for debugging purposes)
        """
        ll    = zeros(8)
        phie  = self.p["phi"] - self.mu_phi0
        err   = (self.X.dot(self.p["phi"]) - self.Y) ** 2
        # p(y)
        ll[0] = - 0.5 * log(2 * pi * self.p["sigma2"]) * self.ndata
        ll[1] = sum(- 0.5 * err / self.p["sigma2"])
        # p(phi)
        ll[2] = - 0.5 * log(2 * pi * self.lbd_phi0 * self.p["sigma2"]) * self.pdata
        ll[3] = - 0.5 * phie.T.dot(phie) / (self.lbd_phi0 * self.p["sigma2"])
        # p(sigma2)
        ll[4] = self.alpha_s20 * log(self.beta_s20)
        ll[5] = - gammaln(self.alpha_s20)
        ll[6] = - (self.alpha_s20 + 1.0) * log(self.p["sigma2"])
        ll[7] = - self.beta_s20 / self.p["sigma2"]
        return sum(ll), ll


    def EM_iter(self):
        """
            Executes a single round of EM updates for this model.

            Has checks to make sure that updates increase logl and
            that parameter values stay in sensible limits.
        """
        # phi
        sumxx         = self.X.T.dot(self.X)
        sumxy         = self.X.T.dot(self.Y)
        sigma_mu      = self.sigma_phi0_inv.dot(self.mu_phi0)
        sigma_phi_inv = self.sigma_phi0_inv + sumxx
#         print("sigma_phi_inv.shape", sigma_phi_inv.shape)
        self.p["phi"] = solve(sigma_phi_inv, sigma_mu + sumxy)
        print("sumxx", type(sumxx), "sumxy", type(sumxy), "sigma_mu", type(sigma_mu), "sigma_phi_inv", type(sigma_phi_inv))
        print("sumxx", sumxx.shape, "sumxy", sumxy.shape, "sigma_mu", sigma_mu.shape, "sigma_phi_inv", sigma_phi_inv.shape)
        self.assert_logl_increased("phi update")

        # sigma2
        phie = (self.p["phi"] - self.mu_phi0) ** 2
        err  = (self.X.dot(self.p["phi"]) - self.Y) ** 2
        num  = self.beta_s20 + 0.5 * sum(err) + 0.5 * sum(phie) / self.lbd_phi0
        den  = self.alpha_s20 + 1.0 + 0.5 * (self.ndata + self.pdata)
        self.p["sigma2"] = num / den
        print("phie", type(phie), "err", type(err), "num", type(num), "den", type(den))
        print("phie", phie.shape, "err", err.shape, "num", num.shape, "den", "is a float")
        if self.p["sigma2"] < 0.0:
            raise ValueError("sigma2 < 0.0")
        self.assert_logl_increased("sigma2 update")


    def print_p(self):
        """
            Prints the model parameters, one at each line.
        """
        print("phi    : %s" % (self.pretty_vector(self.p["phi"])))
        print("sigma2 : %.3f" % (self.p["sigma2"]))



## Generator

In [7]:
# Aalto University, School of Science
# T-61.5140 Machine Learning: Advanced probabilistic Methods
# Author: antti.kangasraasio@aalto.fi, 2016

def generate_X(ndata, pdata):
    """
        Return a matrix of normally distributed random values.
    """
    X = randn(ndata, pdata)
    return X


def generate_YZ(X, distribution):
    """
        Draw observations Y and latent variable values Z from a distribution.
    """
    ndata = len(X)
    Y = zeros(ndata)
    Z = zeros(ndata)
    for i in range(ndata):
        Y[i], Z[i] = distribution.draw(X[i])
    return Y, Z


def get_hyperp():
    """
        Return model hyperparameters.
    """
    return {
            "alpha_s20": 5.0,
            "beta_s20" : 1.0,
            "lbd_phi0" : 1.0,
            "mu_phi0"  : 0.0,
            "alpha_w0" : 3.0,
            "beta_w0"  : 3.0,
            }


## Mixture model

In [21]:
class EM_algo_MM(EM_algo):
    """
        A mixture of two linear models.
    """

    def reset(self):
        """
            Reset priors and draw parameter estimates from prior.
        """
        # priors
        self.alpha_w0       = self.h["alpha_w0"]
        self.beta_w0        = self.h["beta_w0"]

        # Same priors for phi1 and phi2, s2_1, s2_2, don't bother to copy vars twice
        # i.e. alpha_s2_1_0 = alpha_s2_2_0 = alpha_s20
        self.lbd_phi0       = self.h["lbd_phi0"]
        self.alpha_s20      = self.h["alpha_s20"]
        self.beta_s20       = self.h["beta_s20"]
        self.sigma_phi0     = eye(self.pdata) * self.h["lbd_phi0"]
        self.sigma_phi0_inv = eye(self.pdata) / self.h["lbd_phi0"]
        self.mu_phi0        = ones(self.pdata) * self.h["mu_phi0"]
        
        # Precalculations:
        self.w_gamma_ln_multiplier  = gammaln(self.alpha_w0 + self.beta_w0)
        self.w_gamma_ln_multiplier -= gammaln(self.alpha_w0)
        self.w_gamma_ln_multiplier -= gammaln(self.beta_w0)
        
        
        # initial parameter estimates drawn from prior
        self.p             = dict()
        # Weights
        self.p["w"]        = beta(self.alpha_w0, self.beta_w0)
        # Responsibilities (TODO: do we need this here?)
        self.p["gamma"]    = binomial(1, self.p["w"], self.ndata)
        # Component 1
        self.p["sigma2_1"] = 1.0 / gamma(self.alpha_s20, 1.0 / self.beta_s20) # inverse gamma
        self.p["phi_1"]    = mvnormal(self.mu_phi0, self.p["sigma2_1"] * self.sigma_phi0)
        # Component 2
        self.p["sigma2_2"] = 1.0 / gamma(self.alpha_s20, 1.0 / self.beta_s20) # inverse gamma
        self.p["phi_2"]    = mvnormal(self.mu_phi0, self.p["sigma2_2"] * self.sigma_phi0)
        
        print("START w", self.p["w"])
        print("START phi_1", self.p["phi_1"])
        print("START sigma2_1", self.p["sigma2_1"])
        print("START phi_2", self.p["phi_2"])
        print("START sigma2_2", self.p["sigma2_2"])


    def draw(self, item):
        """
            Draw a data sample from the current predictive distribution.
            Returns the y-value and z-value
        """
        mean1 = float(item.dot(self.p["phi_1"]))
        std1  = sqrt(self.p["sigma2_1"])
        mean2 = float(item.dot(self.p["phi_2"]))
        std2  = sqrt(self.p["sigma2_2"])
        
        if np.random.rand() < self.p["w"]:
            return normal(mean1, std1), 1
        else:
            return normal(mean2, std2), 2


    def logl(self):
        """
            Calculates the full log likelihood for this model.
            Returns the logl (and the values of each term for debugging purposes)
        """
        ll         = zeros(20)
        phi_1_diff = self.p["phi_1"] - self.mu_phi0
        phi_2_diff = self.p["phi_2"] - self.mu_phi0
        phi_1_err  = phi_1_diff.T.dot(phi_1_diff)
        phi_2_err  = phi_2_diff.T.dot(phi_2_diff)
        err_1      = (self.X.dot(self.p["phi_1"]) - self.Y) ** 2
        err_2      = (self.X.dot(self.p["phi_2"]) - self.Y) ** 2
        
        # Responsibilities
#         propto_gamma1 =      self.p["w"]  * norm.pdf(self.Y, self.X.dot(self.p["phi_1"]), sqrt(self.p["sigma2_1"]))
#         propto_gamma2 = (1 - self.p["w"]) * norm.pdf(self.Y, self.X.dot(self.p["phi_2"]), sqrt(self.p["sigma2_2"]))
#         gamma = propto_gamma1 / (propto_gamma1 + propto_gamma2)
        gamma = self.p["gamma"]
        
        ### posterior factorizes p(y,z,w,phi,sigma) = p(y)p(z)p(w)p(phi)p(sigma)
        
        ### p(y)
#         ll[0] = self.p["w"]       * (-0.5 * log(2 * pi * self.p["sigma2_1"]) * self.ndata)
#         ll[1] = self.p["w"]       * np.sum(- 0.5 * err_1 / self.p["sigma2_1"])
#         ll[2] = (1 - self.p["w"]) * (-0.5 * log(2 * pi * self.p["sigma2_2"]) * self.ndata)
#         ll[3] = (1 - self.p["w"]) * np.sum(- 0.5 * err_2 / self.p["sigma2_2"])

#         ll[0] =       gamma.dot(-0.5 * log(2 * pi * self.p["sigma2_1"]) -0.5 * err_1 / self.p["sigma2_1"])
#         ll[1] = 0
#         ll[2] = (1 - gamma).dot(-0.5 * log(2 * pi * self.p["sigma2_2"]) -0.5 * err_2 / self.p["sigma2_2"])
#         ll[3] = 0

        ll[0] = -0.5 * log(2 * pi * self.p["sigma2_1"]) * np.sum(gamma)
        ll[1] = gamma.dot(-0.5 * err_1 / self.p["sigma2_1"])
        ll[2] = -0.5 * log(2 * pi * self.p["sigma2_2"]) * np.sum(1 - gamma)
        ll[3] = (1 - gamma).dot(-0.5 * err_2 / self.p["sigma2_2"])
 
        
        ### p(z)
        ll[4] = np.sum((gamma * log(self.p["w"])) + ((1 - gamma) * log(1 - self.p["w"])))
        
        ### p(w)
        ll[5] = self.w_gamma_ln_multiplier
        ll[6] = (self.alpha_w0 - 1) * self.p["w"]
        ll[7] = (self.beta_w0  - 1) * (1 - self.p["w"])
        
        ### p(phi)
        # phi_1
        ll[8]  = - 0.5 * log(2 * pi * self.lbd_phi0 * self.p["sigma2_1"]) * self.pdata
        ll[9]  = - 0.5 * phi_1_err / (self.lbd_phi0 * self.p["sigma2_1"])
        # phi_2
        ll[10] = - 0.5 * log(2 * pi * self.lbd_phi0 * self.p["sigma2_2"]) * self.pdata
        ll[11] = - 0.5 * phi_2_err / (self.lbd_phi0 * self.p["sigma2_2"])
        
        ### p(sigma2)
        # sigma2_1
        ll[12] = self.alpha_s20 * log(self.beta_s20)
        ll[13] = - gammaln(self.alpha_s20)
        ll[14] = - (self.alpha_s20 + 1.0) * log(self.p["sigma2_1"])
        ll[15] = - self.beta_s20 / self.p["sigma2_1"]
        # sigma2_2
        ll[16] = self.alpha_s20 * log(self.beta_s20)
        ll[17] = - gammaln(self.alpha_s20)
        ll[18] = - (self.alpha_s20 + 1.0) * log(self.p["sigma2_2"])
        ll[19] = - self.beta_s20 / self.p["sigma2_2"]
        
        return np.sum(ll), ll


    def EM_iter(self):
        """
            Executes a single round of EM updates for this model.

            Has checks to make sure that updates increase logl and
            that parameter values stay in sensible limits.
        """
        
        Y = self.Y.view()
        Y.shape = (self.ndata, 1) # set Y view shape

        # ==================== E-STEP ====================

        # ========== Responsibilities gamma ==========        
        # norm.pdf works on a vector, returning probability for each separately
        propto_gamma1 =      self.p["w"]  * norm.pdf(self.Y, self.X.dot(self.p["phi_1"]), sqrt(self.p["sigma2_1"]))
        propto_gamma2 = (1 - self.p["w"]) * norm.pdf(self.Y, self.X.dot(self.p["phi_2"]), sqrt(self.p["sigma2_2"]))
        
        # elementwise, works because numpy
        gamma = propto_gamma1 / (propto_gamma1 + propto_gamma2)
        gammaView = gamma.view()
        gammaView.shape = (1, self.ndata)
#         print("gammaView.shape", gammaView.shape)
        self.p["gamma"] = gamma
        sum_gamma = np.sum(gamma)
#         print("sum gamma", sum_gamma, "sum gamma+(1-gamma)", np.sum(gamma+(1-gamma)))
        

        # ==================== M-STEP ====================

        # ========== Component weights w ==========
        num = sum_gamma + self.alpha_w0 - 1
        den = self.ndata + self.alpha_w0 + self.beta_w0 - 2
        self.p["w"] = num / den
        
#         print("W LL DEBUG")
#         newlogl, ll = self.logl()
#         self.debug_logl(self.cll, ll)
#         self.assert_logl_increased("w update")
        
        
        # ========== Variables phi ==========
        
        # phi_1
        sum_gammayx = gamma.T.dot(Y * self.X)
        # NEW
        sum_gammaxx = np.dot( X.T, (gammaView.T * X) )
#         sum_gammaxx = np.dot( X.T, (gammarep.T * X) )
        
#         sum_gammaxx = np.zeros([self.pdata,self.pdata])
#         print("gamma", gamma.shape)
#         for t in range(0,self.ndata):
#             sum_gammaxx += gamma[t] * np.dot(X[t].T, X[t])
# #         sum_gammaxx = gamma.T.dot((self.X * self.X))
        # END NEW
        sigma_phi_inv = self.sigma_phi0_inv + sum_gammaxx
        print("self.sigma_phi0_inv", self.sigma_phi0_inv.shape, "sigma_phi_inv", sigma_phi_inv.shape)
#         print("sum_gammayx, sum_gammaxx SHAPES", sum_gammayx.shape, sum_gammaxx.shape)
        sigma_mu        = self.sigma_phi0_inv.dot(self.mu_phi0)
        sigma_phi_inv   = self.sigma_phi0_inv + sum_gammaxx
        self.p["phi_1"] = solve(sigma_phi_inv, sigma_mu + sum_gammayx)
        print("sum_gammaxx", sum_gammaxx.shape, "sum_gammaxy", sum_gammayx.shape, "sigma_mu", sigma_mu.shape, "sigma_phi_inv", sigma_phi_inv.shape)
        print("phi_1", self.p["phi_1"])
        
        
        # phi_2
        sum_gammayx = (1-gamma).T.dot(Y * self.X)
        # NEW
        sum_gammaxx = np.dot( X.T, ((1-gammaView).T * X) )
#         sum_gammaxx = np.dot( X.T, (one_minus_gammarep.T * X) )
        
#         sum_gammaxx = np.zeros([self.pdata,self.pdata])
#         for t in range(0,self.ndata):
#             sum_gammaxx += (1-gamma[t]) * np.dot(X[t].T, X[t])
#         # END NEW
# #         sum_gammaxx = (1-gamma).T.dot((self.X * self.X))
        print("self.sigma_phi0_inv", self.sigma_phi0_inv.shape, "sigma_phi_inv", sigma_phi_inv.shape)
#         print("sum_gammayx, sum_gammaxx SHAPES", sum_gammayx.shape, sum_gammaxx.shape)
        sigma_mu        = self.sigma_phi0_inv.dot(self.mu_phi0)
        sigma_phi_inv   = self.sigma_phi0_inv + sum_gammaxx
        self.p["phi_2"] = solve(sigma_phi_inv, sigma_mu + sum_gammayx)
        print("sum_gammaxx", sum_gammaxx.shape, "sum_gammaxy", sum_gammayx.shape, "sigma_mu", sigma_mu.shape, "sigma_phi_inv", sigma_phi_inv.shape)
        print("phi_2", self.p["phi_2"])
        
#         print("PHI LL DEBUG")
#         newlogl, ll = self.logl()
#         self.debug_logl(self.cll, ll)
#         self.assert_logl_increased("phi update")


        # ========== Variances sigma2 ==========
        
        # sigma2_1
        phie = np.sum((self.p["phi_1"] - self.mu_phi0) ** 2)  / self.lbd_phi0
        phiX = self.p["phi_1"].dot(self.X.T)
        target_err = (self.Y - phiX)**2
        err_test = (self.X.dot(self.p["phi_1"]) - self.Y) ** 2
        print("err_test", err_test.shape, type(err_test))
        tsup = gamma.dot(err_test)
        err = gamma.dot(target_err)
        print("tsup-err", tsup-err)
        num = 2*self.beta_s20 + err + phie
        den = 2*self.alpha_s20 + 2.0 + np.sum(gamma) + self.pdata
        self.p["sigma2_1"] = num / den
        print("phie", phie.shape, "phiX.shape", phiX.shape, "target_err", target_err.shape, "err", err.shape, "num", num.shape, "den", den.shape)
        if self.p["sigma2_1"] < 0.0:
            raise ValueError("sigma2_1 < 0.0")
        
        # compare ratios
        print("phie/P", phie/self.pdata, "phie", phie, "err/sumGamma", err/np.sum(gamma), "err", err)
        
        
        # sigma2_2
        phie = np.sum((self.p["phi_2"] - self.mu_phi0) ** 2)  / self.lbd_phi0
        phiX = self.p["phi_2"].dot(self.X.T)
        target_err = (self.Y - phiX)**2
        err = (1-gamma).dot(target_err)
        num = 2*self.beta_s20 + err + phie
        den = 2*self.alpha_s20 + 2.0 + np.sum(1-gamma) + self.pdata
        self.p["sigma2_2"] = num / den
        print("phie", phie.shape, "phiX.shape", phiX.shape, "target_err", target_err.shape, "err", err.shape, "num", num.shape, "den", den.shape)
        if self.p["sigma2_2"] < 0.0:
            raise ValueError("sigma2_2 < 0.0")
        
        # compare ratios
        print("phie/P", phie/self.pdata, "phie", phie, "err/sum1-Gamma", err/np.sum(1-gamma), "err", err)
        
        
#         print("SIGMA LL DEBUG")
#         newlogl, ll = self.logl()
#         self.debug_logl(self.cll, ll)
        
#         # if possible, plot samples, true model and estimated model
#         if self.pdata == 1:
#             plt.figure(figsize=(20,10))
#             plt.scatter(self.X, self.Y, s=20, c='black', label="Training data")
#     #         plt.scatter(X_v, Y_v, s=20, c='orange', label="Validation data")
#             x = arange(min(self.X)-0.1, max(self.X)+0.1, 0.1)
#     #         print_linear_model(x, true_model.get_p()["phi"], \
#     #                 true_model.get_p()["sigma2"], 'red', "True model")
#     #         print_linear_model(x, model.get_p()["phi"], \
#     #                 model.get_p()["sigma2"], 'blue', "Predicted model")
#             y = self.p["phi_1"] * x
#             color = 'red'
#             plt.plot(x, y, color, label="component1")
#             plt.fill_between(x, y + 1.96 * sqrt(self.p["sigma2_1"]), y - 1.96 * sqrt(self.p["sigma2_1"]), alpha=0.25, facecolor=color, interpolate=True)
            
#             y = self.p["phi_2"] * x
#             color = 'blue'
#             plt.plot(x, y, color, label="component2")
#             plt.fill_between(x, y + 1.96 * sqrt(self.p["sigma2_2"]), y - 1.96 * sqrt(self.p["sigma2_2"]), alpha=0.25, facecolor=color, interpolate=True)

#             plt.legend(loc=1)
#             plt.xlim(min(x), max(x))
#             plt.xlabel("x")
#             plt.ylabel("y")
#             plt.show()
#             input("Press Enter to continue...")
        
        print("w", self.p["w"], "phi_1", self.p["phi_1"], "phi_2", self.p["phi_2"], "s2_1", self.p["sigma2_1"], "s2_2", self.p["sigma2_2"])
        
        self.assert_logl_increased("sigma2 update")


In [22]:
# get hyperparameters for model
hyperp = get_hyperp()
# generate 50 training data and 20 validation data locations of dim=1
ndata = 200
ndata_v = 50
pdata = 2
X = generate_X(ndata, pdata)
X_v = generate_X(ndata_v, pdata)

true_model = EM_algo_MM(hyperp, ndata=ndata, pdata=pdata)
Y, Z = generate_YZ(X, true_model)
Y_v, Z_v = generate_YZ(X_v, true_model)
print("Generated %d training data and %d validation data from true model:" % \
    (ndata, ndata_v))
true_model.print_p()
print("")

START w 0.25092333255459515
START phi_1 [-0.27081164  0.13296189]
START sigma2_1 0.13098674923551312
START phi_2 [ 0.10960413 -0.03979587]
START sigma2_2 0.1260899189083818
Generated 200 training data and 50 validation data from true model:
gamma = [1 1 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0
 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0
 0 1 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0]
sigma2_1 = 0.13098674923551312
sigma2_2 = 0.1260899189083818
w = 0.25092333255459515
phi_2 = [ 0.10960413 -0.03979587]
phi_1 = [-0.27081164  0.13296189]



In [23]:
# generate a model for estimating the parameters of the
# true model based on the observations (X, Y) we just made
model = EM_algo_MM(hyperp, X, Y)
i, logl, r = model.EM_fit()
print("Model fit (logl %.2f) after %d iterations (%s reached)" % \
        (logl, i, r))
print("")
print("MAP estimate of true model parameters:")
model.print_p()
print("")

# if possible, plot samples, true model and estimated model
if pdata == 1:
    plt.figure(figsize=(20,10))
    plt.scatter(X, Y, s=20, c='black', label="Training data")
#         plt.scatter(X_v, Y_v, s=20, c='orange', label="Validation data")
    x = arange(min(X)-0.1, max(X)+0.1, 0.1)
#         print_linear_model(x, true_model.get_p()["phi"], \
#                 true_model.get_p()["sigma2"], 'red', "True model")
#         print_linear_model(x, model.get_p()["phi"], \
#                 model.get_p()["sigma2"], 'blue', "Predicted model")

    y = true_model.p["phi_1"] * x
    color = 'orange'
    plt.plot(x, y, color, label="true1")
#     plt.fill_between(x, y + 1.96 * sqrt(true_model.p["sigma2_1"]), y - 1.96 * sqrt(true_model.p["sigma2_1"]), alpha=0.1, facecolor=color, interpolate=True)
    
    y = true_model.p["phi_2"] * x
    color = 'green'
    plt.plot(x, y, color, label="true1")
#     plt.fill_between(x, y + 1.96 * sqrt(true_model.p["sigma2_2"]), y - 1.96 * sqrt(true_model.p["sigma2_2"]), alpha=0.1, facecolor=color, interpolate=True)

    # Components
    y = model.p["phi_1"] * x
    color = 'red'
    plt.plot(x, y, color, label="component1")
    plt.fill_between(x, y + 1.96 * sqrt(model.p["sigma2_1"]), y - 1.96 * sqrt(model.p["sigma2_1"]), alpha=0.25, facecolor=color, interpolate=True)

    y = model.p["phi_2"] * x
    color = 'blue'
    plt.plot(x, y, color, label="component2")
    plt.fill_between(x, y + 1.96 * sqrt(model.p["sigma2_2"]), y - 1.96 * sqrt(model.p["sigma2_2"]), alpha=0.25, facecolor=color, interpolate=True)

    plt.legend(loc=1)
    plt.xlim(min(x), max(x))
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()

START w 0.8487573924735687
START phi_1 [-0.07081325 -0.13479411]
START sigma2_1 0.20828230605772718
START phi_2 [ 0.26729434 -0.26509008]
START sigma2_2 0.13372738808935772
gammaView.shape (1, 200)
self.sigma_phi0_inv (2, 2) sigma_phi_inv (2, 2)
sum_gammayx, sum_gammaxx SHAPES (2,) (2, 2)
sum_gammaxx (2, 2) sum_gammaxy (2,) sigma_mu (2,) sigma_phi_inv (2, 2)
phi_1 [-0.03364863  0.00365736]
self.sigma_phi0_inv (2, 2) sigma_phi_inv (2, 2)
sum_gammayx, sum_gammaxx SHAPES (2,) (2, 2)
sum_gammaxx (2, 2) sum_gammaxy (2,) sigma_mu (2,) sigma_phi_inv (2, 2)
phi_2 [ 0.23182242 -0.13230175]
err_test (200,) <class 'numpy.ndarray'>
tsup-err 0.0
phie () phiX.shape (200,) target_err (200,) err () num () den ()
phie/P 0.000572803383814 phie 0.00114560676763 err/sumGamma 0.136470706571 err 23.1910190636
phie () phiX.shape (200,) target_err (200,) err () num () den ()
phie/P 0.0356226935967 phie 0.0712453871935 err/sum1-Gamma 0.0887016014429 err 2.66689844062
w 0.84281394148 phi_1 [-0.03364863  0.00365

ValueError: logl decreased after sigma2 update

## Main

In [17]:
# Aalto University, School of Science
# T-61.5140 Machine Learning: Advanced probabilistic Methods
# Author: antti.kangasraasio@aalto.fi, 2016

def main():
    """
        Executed when program is run.
    """
    print("Starting program")
    print("")
    test_LM_model()


def test_LM_model():
    """
        Example that demonstrates how to call the model.
    """
    # get hyperparameters for model
    hyperp = get_hyperp()
    # generate 50 training data and 20 validation data locations of dim=1
    ndata = 200
    ndata_v = 50
    pdata = 2
    X = generate_X(ndata, pdata)
    X_v = generate_X(ndata_v, pdata)
    # intialize true model randomly and draw observations from it
    true_model = EM_algo_LM(hyperp, ndata=ndata, pdata=pdata)
    Y, Z = generate_YZ(X, true_model)
    Y_v, Z_v = generate_YZ(X_v, true_model)
    print("Generated %d training data and %d validation data from true model:" % \
            (ndata, ndata_v))
    true_model.print_p()
    print("")

    # generate a model for estimating the parameters of the
    # true model based on the observations (X, Y) we just made
    model = EM_algo_LM(hyperp, X, Y)
    i, logl, r = model.EM_fit()
    print("Model fit (logl %.2f) after %d iterations (%s reached)" % \
            (logl, i, r))
    print("")
    print("MAP estimate of true model parameters:")
    model.print_p()
    print("")

    # crossvalidate the estimated model with the validation data
    fit_params = model.get_p()
    model_v = EM_algo_LM(hyperp, X_v, Y_v)
    model_v.set_p(fit_params)
    logl, ll = model_v.logl()
    print("Crossvalidated logl: %.2f" % (logl))

    # if possible, plot samples, true model and estimated model
    if pdata != 1:
        return
    plt.figure(figsize=(20,10))
    plt.scatter(X, Y, s=20, c='black', label="Training data")
    plt.scatter(X_v, Y_v, s=20, c='orange', label="Validation data")
    x = arange(min(X)-0.1, max(X)+0.1, 0.1)
    print_linear_model(x, true_model.get_p()["phi"], \
            true_model.get_p()["sigma2"], 'red', "True model")
    print_linear_model(x, model.get_p()["phi"], \
            model.get_p()["sigma2"], 'blue', "Predicted model")
    plt.legend(loc=1)
    plt.xlim(min(x), max(x))
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()
    


def print_linear_model(x, phi, sigma2, color, label):
    """
        Print linear model mean and 95% confidence interval.
    """
    y = phi * x
    plt.plot(x, y, color, label=label)
    plt.fill_between(x, y + 1.96 * sqrt(sigma2), y - 1.96 * sqrt(sigma2), \
            alpha=0.25, facecolor=color, interpolate=True)


if __name__ == "__main__":
    main()



Starting program

Generated 200 training data and 50 validation data from true model:
phi    : (0.12, -0.08)
sigma2 : 0.147

sumxx <class 'numpy.ndarray'> sumxy <class 'numpy.ndarray'> sigma_mu <class 'numpy.ndarray'> sigma_phi_inv <class 'numpy.ndarray'>
sumxx (2, 2) sumxy (2,) sigma_mu (2,) sigma_phi_inv (2, 2)
phie <class 'numpy.ndarray'> err <class 'numpy.ndarray'> num <class 'numpy.float64'> den <class 'float'>
phie (2,) err (200,) num () den is a float
sumxx <class 'numpy.ndarray'> sumxy <class 'numpy.ndarray'> sigma_mu <class 'numpy.ndarray'> sigma_phi_inv <class 'numpy.ndarray'>
sumxx (2, 2) sumxy (2,) sigma_mu (2,) sigma_phi_inv (2, 2)
phie <class 'numpy.ndarray'> err <class 'numpy.ndarray'> num <class 'numpy.float64'> den <class 'float'>
phie (2,) err (200,) num () den is a float
Model fit (logl -94.43) after 2 iterations (alim reached)

MAP estimate of true model parameters:
phi    : (0.12, -0.07)
sigma2 : 0.152

Crossvalidated logl: -16.43


In [286]:
X = generate_X(10, 2)

In [287]:
X.shape

(10, 2)

In [288]:
X

array([[-0.91954461,  0.2400352 ],
       [ 2.33650195,  1.81126434],
       [-0.08133542,  0.06118832],
       [-1.33545433, -0.86719239],
       [ 1.66719011, -0.57531528],
       [-0.30577782,  0.06323988],
       [ 0.99608716, -0.33705257],
       [-0.65754579, -0.1670884 ],
       [ 1.05556896,  0.3179948 ],
       [ 1.17714858,  1.69832692]])

In [289]:
X[8,:]

array([ 1.05556896,  0.3179948 ])

In [295]:
X.T.dot(X)

array([[ 14.892341  ,   6.29490444],
       [  6.29490444,   7.55600699]])

In [297]:
true_model = EM_algo_LM(get_hyperp(), ndata=10, pdata=2)
Y, Z = generate_YZ(X, true_model)

In [298]:
Y.shape

(10,)

In [299]:
X.shape

(10, 2)

In [300]:
Z

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [301]:
X.T.dot(Y)

array([-6.63206963, -0.67963419])

In [303]:
X.T.dot(X)

array([[ 14.892341  ,   6.29490444],
       [  6.29490444,   7.55600699]])